In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2
import seaborn as sns
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Input
from keras.models import Sequential

In [5]:
image_1_path="/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"
image_2_path="/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor"
image_3_path="/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor"
image_4_path="/kaggle/input/brain-tumor-classification-mri/Training/no_tumor"

In [8]:
image_1_data=[]
image_1_storage = [os.path.join(image_2_path,i) for i in os.listdir(image_2_path)]
image_cv = [ cv2.imread(file) for file in image_1_storage]
# image_array=np.array(image_cv)
for image in image_cv:
    if image is not None and image.size > 0:
        resize_image=cv2.resize(image, (250,250))
        image_1_data.append(resize_image)
        image_array=np.array(image_1_data)

In [9]:
image_array.shape

(822, 250, 250, 3)

In [10]:
def image_convert(image_path):
    import numpy as np
    import cv2
    image_1_data=[]
    image_1_storage = [os.path.join(image_path,i) for i in os.listdir(image_path)]
    image_cv = [ cv2.imread(file) for file in image_1_storage]
    for image in image_cv:
        if image is not None and image.size > 0:
            resize_image=cv2.resize(image, (250,250))
            image_1_data.append(resize_image)
            image_array=np.array(image_1_data)
    return image_array

In [11]:
image_1_data1=image_convert(image_1_path)
image_2_data1=image_convert(image_2_path)
image_3_data1=image_convert(image_3_path)
image_4_data1=image_convert(image_4_path)

In [23]:
image_data_reshaped=image_1_data1.reshape(250*250*3,-1)
image_data_reshaped1=pd.DataFrame(image_data_reshaped)
image_data_reshaped1

,0,1,2,3,4,5,6,7,8,9,...,816,817,818,819,820,821,822,823,824,825
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
3,2,2,2,2,2,2,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
4,2,2,2,2,2,1,1,1,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187495,35,35,36,36,36,34,34,34,36,36,...,32,32,33,33,33,44,44,44,72,72
187496,72,90,90,90,84,84,84,70,70,70,...,58,62,62,62,63,63,63,62,62,62
187497,65,65,65,68,68,68,68,68,68,68,...,5,5,5,5,5,5,5,5,5,4
187498,4,4,4,4,4,4,4,4,4,4,...,4,4,3,3,3,2,2,2,2,2


In [13]:
image_merge=pd.merge(image_1_data1.tolist(),image_2_data1.tolist(),image_3_data1.tolist(),image_4_data1.tolist())

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [ ]:
image_data=np.array(image_1_data)
image_data.shape

In [ ]:
model=Sequential([
    Input(shape=(250,250,3)),
    Conv2D(32,(3,3),activation="relu"),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,(5,5),activation="relu"),
    Conv2D(128,(7,7),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(512,(5,5),activation="relu"),
    Conv2D(128,(7,7),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(1024,(7,7),activation="relu"),
    Conv2D(512,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(2000,activation="relu"),
    Dense(1000,activation="relu"),
    Dropout(0.25),
    Dense(2500,activation="relu"),
    Dense(500,activation="relu"),
    Dropout(0.25),
    Dense(150,activation="relu"),
    Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
# model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(X_train,Y_train,epochs=100,batch_size=32,verbose=1,validation_data=(X_test,Y_test),callbacks=[early_stopping],validation_split=0.2)


In [ ]:
# model.summary()

In [ ]:
q

In [ ]:
# import numpy as np

# # Reshape X_train for compatibility with Conv2D layer
# X_train_array=X_train.values
# X_train_reshaped1 = X_train_array.reshape(-1, 100, 1, 1)
# X_test_array=X_test.values
# X_test_reshaped1=X_test_array.reshape(-1,100,1,1)

In [ ]:
X_test_reshaped1.shape

In [ ]:
model.fit(X_train_reshaped1,Y_train,epochs=100,batch_size=32,verbose=1,validation_data=(X_test_reshaped1,Y_test),callbacks=[early_stopping],validation_split=0.2)


In [ ]:
loss,accuracy=model.evaluate(X_test_reshaped1,Y_test)
print("Loss of Sequential model:",loss)
print("accuracy of Sequential model:",accuracy)

In [ ]:
ref_df2=pd.read_csv("/kaggle/input/ref12-12/ref.csv")

In [ ]:
s=np.array(ref_df2)
s.shape

In [ ]:
values_n1= [
    23.4, 26.7, 30.6, 35.1, 40.0, 44.5, 48.8, 52.9, 56.3, 58.5, 59.7, 60.0, 60.1, 60.6, 61.5, 62.7, 63.6, 63.8, 63.5, 63.1,
    63.2, 63.9, 65.1, 65.6, 65.7, 65.0, 63.5, 60.6, 57.2, 53.5, 49.6, 45.1, 40.0, 34.5, 28.8, 23.3, 18.1, 13.7, 10.3, 7.65,
    5.53, 4.2, 3.43, 3.13, 3.28, 4.2, 5.46, 6.94, 8.51, 10.1, 11.5, 12.7, 13.8, 14.2, 14.0, 12.9, 11.0, 9.05, 7.37, 6.43,
    6.5, 7.33, 8.84, 10.9, 12.9, 14.8, 16.6, 18.4, 20.2, 21.9, 23.5, 23.8, 23.4, 22.5, 21.6, 21.2, 20.8, 20.0, 19.0, 18.3,
    17.8, 17.7, 17.0, 16.0, 14.9, 14.5, 14.8, 15.2, 15.6, 15.4, 14.9, 14.5, 14.2, 14.6, 15.2, 16.0, 16.9, 18.1, 19.6, 21.6
]

In [ ]:
val=np.array(values_n1)

In [ ]:
reshaped_data = val.reshape(1, 100, 1, 1)

In [ ]:
#abnormla=1
values_a1 = [
    29.5, 30.9, 32.6, 34.3, 36.3, 38.5, 40.4, 41.6, 42.8, 44.8, 48.3, 53.6, 60.1, 65.4, 68.2, 69.1, 69.3, 69.3, 69.4, 69.0,
    67.0, 63.6, 59.7, 55.8, 51.8, 47.4, 42.8, 38.3, 34.5, 31.0, 26.7, 21.2, 15.8, 12.0, 9.97, 8.4, 6.52, 4.8, 3.66, 2.85,
    2.1, 1.37, 0.65, 0.27, 0.76, 1.84, 2.7, 3.42, 4.59, 5.56, 5.54, 5.15, 5.0, 4.64, 3.76, 2.76, 2.28, 2.72, 4.0, 5.97,
    8.52, 11.1, 13.1, 14.7, 16.7, 18.8, 20.4, 21.0, 20.9, 20.8, 20.8, 20.9, 20.6, 20.1, 20.0, 20.7, 21.7, 22.3, 22.2, 21.6,
    20.1, 17.3, 14.4, 12.8, 12.7, 13.7, 15.3, 17.2, 18.8, 19.9, 20.6, 21.2, 21.7, 22.1, 22.1, 22.0, 22.5, 23.7, 25.4, 27.0
]

In [ ]:
#normal=0
values_n1= [
    23.4, 26.7, 30.6, 35.1, 40.0, 44.5, 48.8, 52.9, 56.3, 58.5, 59.7, 60.0, 60.1, 60.6, 61.5, 62.7, 63.6, 63.8, 63.5, 63.1,
    63.2, 63.9, 65.1, 65.6, 65.7, 65.0, 63.5, 60.6, 57.2, 53.5, 49.6, 45.1, 40.0, 34.5, 28.8, 23.3, 18.1, 13.7, 10.3, 7.65,
    5.53, 4.2, 3.43, 3.13, 3.28, 4.2, 5.46, 6.94, 8.51, 10.1, 11.5, 12.7, 13.8, 14.2, 14.0, 12.9, 11.0, 9.05, 7.37, 6.43,
    6.5, 7.33, 8.84, 10.9, 12.9, 14.8, 16.6, 18.4, 20.2, 21.9, 23.5, 23.8, 23.4, 22.5, 21.6, 21.2, 20.8, 20.0, 19.0, 18.3,
    17.8, 17.7, 17.0, 16.0, 14.9, 14.5, 14.8, 15.2, 15.6, 15.4, 14.9, 14.5, 14.2, 14.6, 15.2, 16.0, 16.9, 18.1, 19.6, 21.6
]

In [ ]:
values_array = np.array(values_n1)
values_array1 = values_array.reshape(1, 100,1,1)  # Reshape to have batch size 1 and 100 features
predictions = model.predict(values_array1)
binary_predictions = (predictions > 0.5).astype(int)
if binary_predictions[0]==1:
    print("This person is Fatigued")
else:
    print("This person is normal")


In [ ]:
predictions

In [ ]:
binary_predictions

In [ ]:
import numpy as np

# Assuming float32 data type
data_type_bytes = 4  # float32 data type occupies 4 bytes

# Input data size
input_data_shape = (18427, 100, 1)

# Calculate total number of elements
total_elements = np.prod(input_data_shape)

# Calculate total memory usage in bytes
total_memory_bytes = total_elements * data_type_bytes

# Convert bytes to megabytes
total_memory_megabytes = total_memory_bytes / (1024 * 1024)

print("Total GPU memory usage for input data:", total_memory_megabytes, "MB")
